# 📒 Toxicidad ES — Entrenar sin `evaluation_strategy`
Se entrena con `dataset_diverso_10000.csv` y luego se evalúa manualmente (validación y Congreso) **sin usar** el parámetro `evaluation_strategy`.

## 0) Instalar dependencias (si hace falta)

In [ ]:
# Ejecuta SOLO si no tienes estas versiones instaladas o estás en Colab.
!pip -q install -U transformers datasets evaluate scikit-learn accelerate torch pandas matplotlib

## 1) Imports, configuración y rutas

In [3]:
import os, json, random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import evaluate
from datasets import Dataset, DatasetDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay

import torch
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DATA_DIR = Path("../data/processed")
RUNS_DIR = Path("runs")
MODELS_DIR = Path("models")
for d in [DATA_DIR, RUNS_DIR, MODELS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("CUDA disponible:", torch.cuda.is_available())

CUDA disponible: False


## 2) Cargar `dataset_diverso_10000.csv` y crear splits (train/val/test)

In [4]:
BASE_CSV = DATA_DIR / "dataset_diverso_10000.csv"
assert BASE_CSV.exists(), f"No encuentro el archivo: {BASE_CSV}"

df = pd.read_csv(BASE_CSV)

# Detectar columnas de texto/label y normalizar
text_col = next((c for c in df.columns if c.lower() in ["text","texto","sentence"]), None)
label_col = next((c for c in df.columns if c.lower() in ["label","etiqueta","tag"]), None)
assert text_col and label_col, f"No encuentro columnas de texto/label en {df.columns.tolist()}"

df = df.rename(columns={text_col: "texto", label_col: "label"})

def map_label(x):
    x = str(x).strip().lower()
    if x in ["toxico","1","toxic","toxico."]:
        return 1
    return 0
df["label"] = df["label"].map(map_label).astype(int)

train_df, tmp_df = train_test_split(
    df, test_size=0.30, random_state=SEED, stratify=df["label"]
)
val_df, test_df = train_test_split(
    tmp_df, test_size=0.50, random_state=SEED, stratify=tmp_df["label"]
)

train_df.to_csv(DATA_DIR / "train_diverso.csv", index=False)
val_df.to_csv(DATA_DIR / "val_diverso.csv", index=False)
test_df.to_csv(DATA_DIR / "test_diverso.csv", index=False)

train_df.shape, val_df.shape, test_df.shape, train_df['label'].value_counts().to_dict()

((7000, 3), (1500, 3), (1500, 3), {1: 3500, 0: 3500})

## 3) Tokenizar (BETO)

In [5]:
MODEL_ID = "dccuchile/bert-base-spanish-wwm-cased"
MAX_LEN = 256
NUM_LABELS = 2

train = Dataset.from_pandas(pd.read_csv(DATA_DIR / "train_diverso.csv"))
val   = Dataset.from_pandas(pd.read_csv(DATA_DIR / "val_diverso.csv"))
test  = Dataset.from_pandas(pd.read_csv(DATA_DIR / "test_diverso.csv"))

ds = DatasetDict(train=train, validation=val, test=test)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tok_fn(examples):
    return tokenizer(examples["texto"], truncation=True, max_length=MAX_LEN)

ds = ds.map(tok_fn, batched=True)
ds = ds.rename_column("label", "labels")
ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

ds

Map: 100%|██████████| 1500/1500 [00:00<00:00, 74875.11 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'texto', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['id', 'texto', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['id', 'texto', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
})

## 4) *Class weights* (si hay desbalance)

In [6]:
y_train = pd.read_csv(DATA_DIR / "train_diverso.csv")["label"].values
classes = np.unique(y_train)
class_counts = np.bincount(y_train)
print("Distribución de clases (train):", dict(enumerate(class_counts)))

total = class_counts.sum()
class_weights = torch.tensor([total / (len(classes) * c) if c > 0 else 0.0 for c in class_counts], dtype=torch.float)
class_weights

Distribución de clases (train): {0: np.int64(3500), 1: np.int64(3500)}


tensor([1., 1.])

## 5) Modelo y `Trainer` (sin `evaluation_strategy`)

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=NUM_LABELS)

metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

class WeightedTrainer(Trainer):
    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: int | None = None,   # <-- clave para versiones nuevas
        **kwargs,                                 # <-- y por si cambian algo más
    ):
        # No mutar inputs originales
        labels = inputs["labels"]
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}

        outputs = model(**model_inputs)
        logits = outputs.logits

        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


# 🔧 No usamos evaluation_strategy ni load_best_model_at_end
args = TrainingArguments(
    output_dir=str(RUNS_DIR / "beto-toxic-diverso-no-evalstrategy"),
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=3,
    # save_strategy='no',  # opcional: sin checkpoints
    logging_steps=50,
    report_to="none",
    seed=SEED
)

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    # eval_dataset omitido para evitar cualquier evaluación durante train
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\fernando\AppData\Local\Temp\ipykernel_21476\1309764010.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


## 6) Entrenamiento

In [10]:
train_result = trainer.train()
train_result.metrics

Step,Training Loss
50,0.105700
100,0.000400
150,0.000300
200,0.000200
250,0.000100
300,0.000100
350,0.000100
400,0.000100
450,0.000100
500,0.000100


c:\Users\fernando\OneDrive\Documentos\Proyectos\NLP\nlp-toxicidad-parlamentaria\.env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'train_runtime': 965.0561,
 'train_samples_per_second': 21.76,
 'train_steps_per_second': 1.362,
 'total_flos': 272303497856640.0,
 'train_loss': 0.004102648006659285,
 'epoch': 3.0}

## 7) Evaluación manual en validación y test (después del entrenamiento)

In [11]:
# Validación
eval_val = trainer.evaluate(ds["validation"])
print("Métricas (validación):", eval_val)

# Test
eval_test = trainer.evaluate(ds["test"])
print("Métricas (test):", eval_test)

# Reporte en test
pred_out = trainer.predict(ds["test"])
logits = pred_out.predictions
y_true = pred_out.label_ids
y_pred = logits.argmax(-1)

print("\nClassification report (test-diverso):\n")
print(classification_report(y_true, y_pred, digits=4))

c:\Users\fernando\OneDrive\Documentos\Proyectos\NLP\nlp-toxicidad-parlamentaria\.env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Métricas (validación): {'eval_loss': 1.7539823602419347e-05, 'eval_accuracy': 1.0, 'eval_f1_macro': 1.0, 'eval_f1_weighted': 1.0, 'eval_runtime': 13.4982, 'eval_samples_per_second': 111.126, 'eval_steps_per_second': 3.482, 'epoch': 3.0}


c:\Users\fernando\OneDrive\Documentos\Proyectos\NLP\nlp-toxicidad-parlamentaria\.env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Métricas (test): {'eval_loss': 1.7515505533083342e-05, 'eval_accuracy': 1.0, 'eval_f1_macro': 1.0, 'eval_f1_weighted': 1.0, 'eval_runtime': 11.6756, 'eval_samples_per_second': 128.473, 'eval_steps_per_second': 4.025, 'epoch': 3.0}


c:\Users\fernando\OneDrive\Documentos\Proyectos\NLP\nlp-toxicidad-parlamentaria\.env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Classification report (test-diverso):

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       750
           1     1.0000    1.0000    1.0000       750

    accuracy                         1.0000      1500
   macro avg     1.0000    1.0000    1.0000      1500
weighted avg     1.0000    1.0000    1.0000      1500



## 8) Guardar modelo, tokenizer y métricas

In [ ]:
SAVE_DIR = MODELS_DIR / "beto-toxicidad-diverso-no-evalstrategy"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

trainer.save_model(str(SAVE_DIR))
tokenizer.save_pretrained(str(SAVE_DIR))

with open(SAVE_DIR / "metrics_test_diverso.json", "w") as f:
    json.dump({k: float(v) for k,v in eval_test.items()}, f, indent=2)

print("Guardado en:", SAVE_DIR)

## 9) Evaluación *out-of-domain* en `intervenciones_2020_17.csv` (Congreso)

In [ ]:
CONG_CSV = DATA_DIR / "intervenciones_2020_17.csv"
assert CONG_CSV.exists(), f"No encuentro el archivo: {CONG_CSV}"

df_eval = pd.read_csv(CONG_CSV)

text_col = next((c for c in df_eval.columns if c.lower() in ["text","texto","sentence"]), None)
label_col = next((c for c in df_eval.columns if c.lower() in ["label","etiqueta","tag"]), None)
assert text_col and label_col, f"No encuentro columnas de texto/label en {df_eval.columns.tolist()}"

df_eval = df_eval.rename(columns={text_col: "texto", label_col: "label"})

def map_label_eval(x):
    s = str(x).strip().lower()
    if s in ["toxico","1","toxic","toxico."]:
        return 1
    if s in ["no_toxico","0","not_toxic","no-toxico"]:
        return 0
    try:
        return int(float(s))
    except:
        return 0
df_eval["label"] = df_eval["label"].map(map_label_eval).astype(int)

ds_eval = Dataset.from_pandas(df_eval)
ds_eval = ds_eval.map(lambda ex: tokenizer(ex["texto"], truncation=True, max_length=MAX_LEN), batched=True)
ds_eval = ds_eval.rename_column("label", "labels")
ds_eval.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

eval_congreso = trainer.evaluate(ds_eval)
print("Métricas (Congreso OOD):", eval_congreso)

pred_out_c = trainer.predict(ds_eval)
logits_c = pred_out_c.predictions
y_true_c = pred_out_c.label_ids
y_pred_c = logits_c.argmax(-1)

print("\nClassification report (Congreso):\n")
print(classification_report(y_true_c, y_pred_c, digits=4))

cm = confusion_matrix(y_true_c, y_pred_c)
fig = plt.figure(figsize=(4,4))
plt.imshow(cm, interpolation='nearest')
plt.title("Matriz de confusión — Congreso")
plt.xticks([0,1], ["No tóxico","Tóxico"])
plt.yticks([0,1], ["No tóxico","Tóxico"])
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

## 10) Inferencia rápida

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

clf = AutoModelForSequenceClassification.from_pretrained(SAVE_DIR)
tok = AutoTokenizer.from_pretrained(SAVE_DIR)
clf.eval()

def predict_toxicidad(textos, batch_size=32, threshold=0.5):
    if isinstance(textos, str):
        textos = [textos]
    all_scores = []
    with torch.no_grad():
        for i in range(0, len(textos), batch_size):
            batch = textos[i:i+batch_size]
            enc = tok(batch, truncation=True, max_length=MAX_LEN, padding=True, return_tensors="pt")
            if torch.cuda.is_available():
                enc = {k: v.cuda() for k, v in enc.items()}
                clf.cuda()
            out = clf(**enc).logits
            proba = torch.softmax(out, dim=1).cpu().numpy()[:,1]
            all_scores.extend(proba.tolist())
    preds = (np.array(all_scores) >= threshold).astype(int).tolist()
    return {"y_pred": preds, "score_tox": all_scores}

predict_toxicidad([
    "Se ruega respeto en esta sala, no toleraremos insultos.",
    "Cállate de una vez, ignorante."
])